<a href="https://colab.research.google.com/github/RodrigoSalles/Big-Data-and-Cloud-Computing---Colab/blob/master/Google_Cloud_Interface.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Google Cloud interface

**[Big Data and Cloud Computing](https://www.dcc.fc.up.pt/~edrdo/aulas/bdcc), Eduardo R. B. Marques, DCC/FCUP**

This notebook explains how to setup automatic authentication for Google Cloud APIs and Google Cloud SDK utilities (e.g., `gcloud` or `gsutil`).

## Generating a service account and key file

We first need to obtain a __key file__ for a GCP __service account__. 

- A service account is associated with a GCP project, letting you authenticate and access GCP resources in the project from external code, including Google Colab.
- A key file plays the role of an authentication token.  

The instructions are a summary of the [Getting Started with Authentication](https://cloud.google.com/docs/authentication/getting-started) GCP guide. We will use the Google Cloud Shell to generate the service key file.

### Access the Google Cloud Shell

Access the [Google Cloud Console](https://console.cloud.google.com/),make sure your project is selected, then click the Google Cloud Shell button.

![](https://drive.google.com/uc?id=1hk2LADT_GimQgOtKPfENZ1Jbd6SUKSr9)

![](https://drive.google.com/uc?id=16DACIte8f6ipWos09njBhkgZDqJnBrzA)

### Create service account and key file

Issue the following commands in sequence (a summary of 
the [Getting Started with Authentication](https://cloud.google.com/docs/authentication/getting-started) GCP guide):

1. Parameterize the name of the project (__P__) and the name of the service account (__S__) as environment variables (for convenience).
  
  ```
  P=NAME_OF_YOUR_PROJECT
  S=bdcc-colab
  ```

2. Create the service account

  ```
gcloud iam service-accounts create $S
  ````

3. Grant permissions to the service account.
  
  ```
gcloud projects add-iam-policy-binding "$P" --member "serviceAccount:$S@$P.iam.gserviceaccount.com" --role "roles/owner"
  ```
4. Generate the key file, in this case named `bdcc-colab.json`.
  
  ```
gcloud iam service-accounts keys create bdcc-colab.json --iam-account "$S@$P.iam.gserviceaccount.com"
  ```

5. Download the key file.
  ```
cloudshell download bdcc-colab.json
  ```

The steps and corresponding GCP output are illustrated below:

```
eduardorbmarques@cloudshell:~ (bdcc20)$ P=bdcc20
eduardorbmarques@cloudshell:~ (bdcc20)$ S=bdcc-colab
eduardorbmarques@cloudshell:~ (bdcc20)$ gcloud iam service-accounts create $S
Created service account [bdcc-colab].
eduardorbmarques@cloudshell:~ (bdcc20)$ gcloud projects add-iam-policy-binding "$P" --member "serviceAccount:$S@$P.iam.gservicea
ccount.com" --role "roles/owner"
Updated IAM policy for project [bdcc20].
bindings:
- members:
  - serviceAccount:service-560713482346@gcf-admin-robot.iam.gserviceaccount.com
  role: roles/cloudfunctions.serviceAgent
- members:
  - serviceAccount:service-560713482346@compute-system.iam.gserviceaccount.com
  role: roles/compute.serviceAgent
- members:
  - serviceAccount:service-560713482346@container-engine-robot.iam.gserviceaccount.com
  role: roles/container.serviceAgent
- members:
  - serviceAccount:560713482346-compute@developer.gserviceaccount.com
  - serviceAccount:560713482346@cloudservices.gserviceaccount.com
  - serviceAccount:bdcc20@appspot.gserviceaccount.com
  - serviceAccount:service-560713482346@containerregistry.iam.gserviceaccount.com
- members:
  - serviceAccount:560713482346-compute@developer.gserviceaccount.com
  - serviceAccount:560713482346@cloudservices.gserviceaccount.com
  - serviceAccount:bdcc20@appspot.gserviceaccount.com
  - serviceAccount:service-560713482346@containerregistry.iam.gserviceaccount.com
  role: roles/editor
- members:
  - serviceAccount:bdcc-colab@bdcc20.iam.gserviceaccount.com
  - serviceAccount:bdccservice@bdcc20.iam.gserviceaccount.com
  - user:eduardorbmarques@gmail.com
  role: roles/owner
etag: BwWfPaTh12A=
version: 1
eduardorbmarques@cloudshell:~ (bdcc20)$ gcloud iam service-accounts keys create bdcc-colab.json --iam-account "$S@$P.iam.gserviceaccount.com"
created key [95f02d6f6ee98187d2e5359691d498947727f8de] of type [json] as [bdcc-colab.json] for [bdcc-colab@bdcc20.iam.gserviceaccount.com]
eduardorbmarques@cloudshell:~ (bdcc20)$ cloudshell download bdcc-colab.json
```

## Use the credentials in Colab

1. Upload the key file onto Google Drive (use the Google drive interface).
2. Mount the Google Drive in this notebook.
3. Assuming that the key file is stored in `/content/drive/My Drive/bdcc-colab.json` set up the `GOOGLE_APPLICATION_CREDENTIALS` environment variable by running the following cell.




In [0]:
import os
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = '/content/drive/My Drive/bdcc-colab.json' 
!echo $GOOGLE_APPLICATION_CREDENTIALS

### Google Cloud APIs

The environment variable setting is enough for authentication when using several Google Cloud  APIs. For example we may use the [Google Cloud Storage API](https://googleapis.dev/python/storage/latest/client.html), as shown below:

In [0]:
from google.cloud import storage

storage_client = storage.Client()

buckets = storage_client.list_buckets()

print('-- List of buckets in project \"' + storage_client.project + '\"')

for b in buckets:
  print(b.name)



### Google Cloud SDK utilities

For using the Google Cloud SDK utilities like `gcloud` or `gsutil`, we need to perform the following command first:

In [0]:
import os

project='bdcc20' # change to your project name here
os.environ['GCP_PROJECT'] = project 
os.environ['GCP_ACCOUNT'] = 'bdcc-colab@' + project + '.iam.gserviceaccount.com'

!gcloud auth activate-service-account "$GCP_ACCOUNT" --key-file="$GOOGLE_APPLICATION_CREDENTIALS" --project="$GCP_PROJECT"

We can now use `gcloud` and  `gsutil`.

In [0]:
!gcloud info

... and `gsutil` as well:

In [0]:
!gsutil ls